![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_RoBERTa.ipynb)

# Import ONNX RoBERTa models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for RoBERTa from HuggingFace and they have to be in `Fill Mask` category. Meaning, you cannot use RoBERTa models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.34.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.34.1 optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.3/252.3 kB 16.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.2/451.2 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 104.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 101

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [roberta-base](https://huggingface.co/roberta-base) model from HuggingFace as an example and load it as a `ORTModelForFeatureExtraction`, representing an ONNX model.
- In addition to the RoBERTa model, we also need to save the tokenizer. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [ ]:
from optimum.onnxruntime import ORTModelForFeatureExtraction

MODEL_NAME = "roberta-base"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForFeatureExtraction.from_pretrained(MODEL_NAME, export=True)

# Save the ONNX model
ort_model.save_pretrained(EXPORT_PATH)

Framework not specified. Using pt to export to ONNX.
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using framework PyTorch: 2.0.1+cu118
Overriding 1 configuration item(s)
	- use_cache -> False


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
# Create directory for assets
!mkdir {EXPORT_PATH}/assets

# let's make sure we sort the vocabs based on their ids first
vocabs = ort_model.preprocessors[0].get_vocab()
vocabs = sorted(vocabs, key=vocabs.get)

# let's save the vocab as txt file
with open(f'{EXPORT_PATH}/vocab.txt', 'w') as f:
    for item in vocabs:
        f.write("%s\n" % item)

# let's copy both vocab.txt and merges.txt files to saved_model/1/assets
!cp {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets
!cp {EXPORT_PATH}/merges.txt {EXPORT_PATH}/assets

Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {EXPORT_PATH}

total 488536
drwxr-xr-x 2 root root      4096 Jul 11 10:38 assets
-rw-r--r-- 1 root root       644 Jul 11 10:37 config.json
-rw-r--r-- 1 root root    456318 Jul 11 10:37 merges.txt
-rw-r--r-- 1 root root 496467018 Jul 11 10:37 model.onnx
-rw-r--r-- 1 root root       280 Jul 11 10:37 special_tokens_map.json
-rw-r--r-- 1 root root       351 Jul 11 10:37 tokenizer_config.json
-rw-r--r-- 1 root root   2108619 Jul 11 10:37 tokenizer.json
-rw-r--r-- 1 root root    798293 Jul 11 10:37 vocab.json
-rw-r--r-- 1 root root    407065 Jul 11 10:38 vocab.txt


In [ ]:
!ls -l {EXPORT_PATH}/assets

total 848
-rw-r--r-- 1 root root 456318 Jul 11 10:38 merges.txt
-rw-r--r-- 1 root root 407065 Jul 11 10:38 vocab.txt


## Import and Save RoBERTa in Spark NLP

- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.0.0
setup Colab for PySpark 3.2.3 and Spark NLP 5.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.7/498.7 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 17.3 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `BertEmbeddings` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `BertEmbeddings` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want!
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [ ]:
from sparknlp.annotator import *

# All these params should be identical to the original ONNX model
roberta = RoBertaEmbeddings.loadSavedModel(f"{EXPORT_PATH}", spark)\
    .setInputCols(["document",'token'])\
    .setOutputCol("roberta")\
    .setCaseSensitive(True)\
    .setStorageRef('roberta-base')

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
roberta.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your ONNX RoBERTa model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

total 484920
drwxr-xr-x 4 root root      4096 Jul 11 10:43 fields
drwxr-xr-x 2 root root      4096 Jul 11 10:43 metadata
-rw-r--r-- 1 root root 496542911 Jul 11 10:43 roberta_onnx


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny RoBERTa model 😊

In [ ]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *

document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")

roberta_loaded = RoBertaEmbeddings.load(f"{MODEL_NAME}_spark_nlp")\
    .setInputCols(["document",'token'])\
    .setOutputCol("roberta")\

pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        roberta_loaded
  ])

data = spark.createDataFrame([['William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor,and philanthropist.']]).toDF("text")
model = pipeline.fit(data)
result = model.transform(data)

In [ ]:
result.selectExpr("explode(roberta.embeddings) as embeddings").show()

+--------------------+
|          embeddings|
+--------------------+
|[0.38130012, -0.5...|
|[0.44632936, -0.2...|
|[0.51096636, 0.08...|
|[-0.35451564, -0....|
|[0.39058837, 0.43...|
|[-0.49727106, 0.9...|
|[0.16452815, -0.4...|
|[0.17555067, -0.2...|
|[-0.024700366, 0....|
|[-0.3068737, 0.37...|
|[0.58636177, 0.59...|
|[-0.6334504, 0.09...|
|[-0.6704677, -0.3...|
|[-0.046930477, -0...|
|[0.15983628, -0.4...|
|[0.4974299, -0.17...|
|[1.1059247, 0.549...|
|[-0.19082792, -0....|
|[-0.17161375, -0....|
|[0.71347564, 0.23...|
+--------------------+
only showing top 20 rows



That's it! You can now go wild and use hundreds of RoBERTa models from HuggingFace 🤗 in Spark NLP 🚀
